<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Third Model: Deep Learning
Your first model took random shots and your second model tried matching the current game against past games; however, if no past games matched then it just randomly guessed. This model will do better by __learning__ the pattern of ship layouts, not just memorizing them. 

## Data
We will be using past game date to train out model. First we need to download the data by running: (note the data is constantly being updated as games are played)
```bash
./bin/download_data.sh
```
next copy the sample data file into the mock directory of the code for this model
```bash
cp ./data/data.min.json ./containers/Shoot/cnn/mock/data/data.json
```
You will now be able to test your model training localy 

Next we need to upload the data to s3 so we can train the production model on it. run:
```bash
./bin/upload_data.sh
```

## Code
1. code is in /containers/shoot/cnn


## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in /deploy-config.js. you need to edit the following field to the following values

|parameter field|Value|description|
|---|---|
|trainsourcefile|`shoot-CNN.tar.gz`|
|hostsourcefile|`shoot-CNN.tar.gz`|
|hyperparameters.epochs|200|
|hyperparameters.learning_rate|0.005|
|hyperparameters.width|32|
|hyperparameters.depth|8|
|hyperparameters.patience|10|
|hyperparameters.batch_size|1024|
|metrics|[{Name:"Validation",Regex:"Testing loss: (.*?);"},{Name:"Training",Regex:"Training loss: (.*?);"},{Name:"Throughput",Regex:"Throughput=(.*?);"}]|
|traininstancetype|ml.p3.2xlarge|
|channels|{train:{path:"all/data.json"}}|

## Deploy
1. follow instructions from endpoint reference to commit your code and changes and then push your changes to the remote branch
2. tell opperations to deploy your changes

## Next Steps
try editing hyperparamters to get better results